# 5.4 독립형 질문 챗봇 만들기
- 라이브러리: langchain, streamlit, PyPDF2
- 언어모델: gpt-3.5-turbo
- 임베딩모델: all_MiniLM-L6-v2
- 벡터 데이터베이스: 파이스(FAISS)
  

## 코드 흐름
> 내가 가지고 있는 PDF 파일을 이용하여 RAG를 구성  
> (질문을 받으면 먼저 내가 가지고 있는 PDF파일에서 내용을 찾아보고 결과를 llm에 제공하여 답변을 하는 방식)

In [1]:
import streamlit as st
from PyPDF2 import PdfReader
from langchain.embeddings import OpenAIEmbeddings, SentenceTransformerEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.memory import ConversationBufferWindowMemory
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
# PDF 문서에서 텍스트를 추출
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

# 지정된 조건에 따라 주어진 텍스트를 더 작은 덩어리로 분할
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(
        # separator는 청크 분할 기준, chunk_size는 청크 크기, chunk_overlap은 청크간 중복 크기
        separators = "\\n",
        chunk_size = 1000,
        chunk_overlap = 200,
        length_function = len
    )
    chunk = text_splitter.split_text(text)
    return chunk

# 주어진 텍스트 청크에 대한 임베딩을 생성하고 파이스(FAISS)를 사용하여 벡터 저장소를 생성.
def get_vectorstore(text_chunks):
    # 임베딩 처리(벡터 변환), HuggingFaceEmbeddings 모델 사용
    embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore

In [4]:
# 주어진 벡터 저장소로 대화 체인을 초기화
def get_conversation_chain(vectorstore):
    # ConversationBufferWindowsMemory에 이전 대화 저장
    # ConversationBufferWindowMemory를 사용하여 이전 대화의 기록을 관리합니다.
    # memory_key는 메모리 내에서 대화 내용을 저장하는 키 이름입니다.
    # return_message는 대화 내용을 반환할지 여부를 설정합니다.
    memory = ConversationBufferWindowMemory(memory_key='chat_history', return_message=True)
    
    # ConversationRetrievalChain을 통해 랭체인 쳇봇에 쿼리 전송
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm = ChatOpenAI(temperature=0, model_name = 'gpt-3.5-turbo'),
        # retriever는 벡터 저장소를 검색 엔진으로 사용하여 쿼리와 가장 관련있는 내용을 검색합니다.
        retriever = vectorstore.as_retriever(),
        # get_chat_history는 대화의 역사를 관리하는 함수를 지정합니다. 여기서는 전달된 인자를 그대로 반환합니다.
        get_chat_history = lambda h: h,
        # memory는 위에서 설정한 대화 내용 기록 메모리를 사용합니다.
        memory = memory
    )
    return conversation_chain

In [5]:
user_uploads = st.file_uploader("파일을 업로드해 주세요~", accept_multiple_files=True)

if user_uploads is not None:
    if st.button("Upload"):
        with st.spinner("차리중.."):
            # PDF 텍스트 가져오기
            raw_text = get_pdf_text(user_uploads)
            # 텍스트에서 청크 검색
            text_chunks = get_text_chunks(raw_text)
            # PDF 텍스트 저장을 위해 파이스(FAISS) 벡터 저장소 만들기
            vectorstore = get_vectorstore(text_chunks)
            # 대화 체인 만들기
            st.session_state.conversation = get_conversation_chain(vectorstore)

2024-09-20 12:29:10.240 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 12:29:10.240 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 12:29:10.243 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 12:29:10.338 
  command:

    streamlit run /opt/anaconda3/envs/llm_project/lib/python3.8/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-09-20 12:29:10.339 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 12:29:10.339 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 12:29:10.339 Thread 'MainThread': missing ScriptRunContext! This warning can be 

In [7]:
if user_query := st.chat_input("질문을 입력해주세요~"):
    # 대화 체인을 사용하여 사용자의 메시지를 처리
    if 'conversation' in st.session_state:
        result = st.session_state.conversation({
            "question": user_query,
            "chat_history": st.session_state.get('chat_history', [])
        })
        response = result["answer"]
    else:
        response = "먼저 문서를 업로드해주세요~."
    with st.chat_message("assistant"):
        st.write(response)


2024-09-20 12:29:16.900 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 12:29:16.901 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 12:29:16.902 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 12:29:16.903 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
